In [ ]:
!pip install gymnasium
from stable_baselines3.common.env_checker import check_env
import gymnasium as gym
from gym import Env
from gym import spaces
import random
import numpy as np
import os


In [ ]:
NOTHING = 0
PLAYER = 1
WIN = 2
LOSE = 3
UP = 0
DOWN = 1
LEFT = 2
RIGHT = 3

In [ ]:
class BasicEnv(gym.Env):
    def __init__(self):
        self.cumulative_reward = 0
        #
        # set the initial state to a flattened 6x6 grid with a randomly placed entry, win, and player
        #
        self.state = [NOTHING] * 36

        self.player_position = random.randrange(0, 36)
        self.win_position = random.randrange(0, 36)
        self.lose_position = random.randrange(0, 36)

        # make sure the entry and lose points aren't overlapping eachother
        while self.win_position == self.player_position:
            self.win_position = random.randrange(0, 36)

        while self.lose_position == self.win_position or self.lose_position == self.player_position:
            self.lose_position = random.randrange(0, 36)

        self.state[self.player_position] = PLAYER
        self.state[self.win_position] = WIN
        self.state[self.lose_position] = LOSE

        # convert the python array into a numpy array (needed since Gym expects the state to be this way)
        self.state = np.array(self.state, dtype=np.int16)

        # observation space (valid ranges for observations in the state)
        self.observation_space = gym.spaces.Box(0, 3, [36,], dtype=np.int16)

        # valid actions:
        #   0 = up
        #   1 = down
        #   2 = left
        #   3 = right
        self.action_space = gym.spaces.Discrete(4)

    def step(self, action):
        # placeholder for debugging information
        info = {}

        # set default values for done, reward, and the player position before taking the action
        done = False
        reward = -0.01
        previous_position = self.player_position

        #
        # take the action by moving the player
        #
        if action == UP:
            if (self.player_position - 6) >= 0:
                self.player_position -= 6

        elif action == DOWN:
            if (self.player_position + 6) < 36:
                self.player_position += 6

        elif action == LEFT:
            if (self.player_position % 6) != 0:
                self.player_position -= 1

        elif action == RIGHT:
            if (self.player_position % 6) != 5:
                self.player_position += 1
        else:
            raise Exception("invalid action")

        #
        # check for win/lose conditions and set reward
        #
        if self.state[self.player_position] == WIN:
            reward = 1.0
            self.cumulative_reward += reward
            done = True
            clear_screen()
            print(f'Cumulative Reward: {self.cumulative_reward}')
            print('YOU WIN!!!!')

        elif self.state[self.player_position] == LOSE:
            reward = -1.0
            self.cumulative_reward += reward
            done = True
            clear_screen()
            print(f'Cumulative Reward: {self.cumulative_reward}')
            print('YOU LOSE')

        #
        # Update the environment state
        #
        if not done:
            # update the player position
            self.state[previous_position] = NOTHING
            self.state[self.player_position] = PLAYER

        self.cumulative_reward += reward
        truncated=False
        return self.state, reward, done, truncated,info

    def render(self):
        # visualization can be added here
        pretty_print(self.state, self.cumulative_reward)

    def reset(self,seed=None):
        if seed is not None:
          random.seed(seed)
        self.cumulative_reward = 0
        #
        # set the initial state to a flattened 6x6 grid with a randomly placed entry, win, and player
        #
        self.state = [NOTHING] * 36

        self.player_position = random.randrange(0, 36)
        self.win_position = random.randrange(0, 36)
        self.lose_position = random.randrange(0, 36)

        # make sure the entry and lose points aren't overlapping eachother
        while self.win_position == self.player_position:
            self.win_position = random.randrange(0, 36)

        while self.lose_position == self.win_position or self.lose_position == self.player_position:
            self.lose_position = random.randrange(0, 36)

        self.state[self.player_position] = PLAYER
        self.state[self.win_position] = WIN
        self.state[self.lose_position] = LOSE

        # convert the python array into a numpy array (needed since Gym expects the state to be this way)
        self.state = np.array(self.state, dtype=np.int16)
        observation=self.state
        info={}
        return observation,info

In [ ]:
def pretty_print(state_array, cumulative_reward):
    clear_screen()
    print(f'Cumulative Reward: {cumulative_reward}')
    print()
    for i in range(6):
        for j in range(6):
            print('{:4}'.format(state_array[i*6 + j]), end = "")
        print()

def clear_screen():
    clear_output()
    os.system("cls")


In [ ]:
1#from BasicEnvironment import *

env = BasicEnv()
env.render()
action = int(input("Enter action:"))
state, reward, done,truncated, info = env.step(action)
while not done:
    env.render()
    action = int(input("Enter action:"))
    state, reward, done,truncated, info = env.step(action)

Cumulative Reward: 0.99
YOU WIN!!!!


In [ ]:
from IPython.display import clear_output

In [ ]:
!pip install stable_baselines3

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
#from BasicEnvironment import *
from stable_baselines3.common.env_checker import check_env
env = BasicEnv()
check_env(env)

Cumulative Reward: 1.0
YOU WIN!!!!


In [ ]:
episodes = 50
for episode in range(episodes):
  done=False
  obs=env.reset()
  i=0
  while(i<=50):
    random_action=env.action_space.sample()
    env.render()
    print("action",random_action)
    obs,reward,done,truncated,info=env.step(random_action)
    print('reward',reward)
    i+=1

Cumulative Reward: 1.99

   0   0   0   0   0   0
   0   0   0   0   0   0
   0   0   0   1   2   0
   0   0   0   0   0   0
   3   0   0   0   0   0
   0   0   0   0   0   0
action 3
reward -0.01


KeyboardInterrupt: 